In [1]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import io
import pandas as pd
import threading
import time
import os

In [2]:
kafka_broker = 'localhost:29092'

In [ ]:
consumer = KafkaConsumer(
        'data_extracted',
        bootstrap_servers=[kafka_broker],
        enable_auto_commit=True,
        value_deserializer=lambda x: x.decode('utf-8')
    )
producer = KafkaProducer(
        bootstrap_servers=[kafka_broker],
        linger_ms=5000,
        acks='all',
        max_block_ms=60000
    )

: 

In [ ]:
import threading
import time
import os
import pandas as pd


def consume_and_send():
    output_file_path = 'received_file.csv'
    print('Running Consumer')

    try:
        for message in consumer:
            print(f"Received message: [{message.timestamp}:{message.offset}] {message.value}")

            # Write the received message (CSV content) to a file
            with open(output_file_path, 'w') as f:
                f.write(message.value)  # Save the content to the file

            print(f"Message saved to {output_file_path}")

            # Read the CSV file
            df = pd.read_csv(output_file_path)

            # Data cleaning steps
            df.drop(['authors', 'article_info', 'citation_info', 'document_info', 'affiliations', 'funding', 'abstract'], axis=1, inplace=True)
            df.dropna()  # Drop rows with missing values
            df.dropna(axis=1, inplace=True)  # Drop columns with missing values
            df.drop_duplicates(inplace=True)

            # Modify column names
            df.columns = df.columns.str.lower().str.replace(' ', '_')

            # Apply lowercase to all string values
            df = df.apply(lambda col: col.str.lower() if col.dtype == "object" else col)

            # Save cleaned data to a new CSV
            df.to_csv('cleaned_data.csv', index=False)
            with open('cleaned_data.csv', 'r') as csvfile:
                csv_content = csvfile.read()  # Read entire file content


            print(f"Sending CSV content to Kafka: {csv_content[:100]}...")  # Log first 100 characters
            producer.send('processed_data', csv_content.encode('utf-8'))
            time.sleep(2)
            producer.flush()


    except Exception as e:
        print(f"Error while consuming: {e}")

try:
    consume_and_send()
except Exception as e:
  print(f"Error while calling: {e}")
#         retry_count += 1  # Increment retry count for unexpected errors

Running Consumer
Received message: [1733422890422:25] title,authors,article_info,abstract,categories/keyword,citation_info,document_info,author_tags,affiliations,funding
Incidence and risk factors associated with injuries during static line parachute training in Royal Thai Army(Article)(Open Access),"['Maneechaeye, W.', 'Deepreecha, K.', 'Jiamjarasrangsi, W.']","Volume 7, Issue 1, 7 June 2020, Article number 27","Background: Incidence and risk factors of parachute injuries has been studied in developed countries, but not in trainees of the airborne forces in the Royal Thailand Army. Methods: A prospective cohort study was conducted among 992 military personnel who attended the basic airborne training program from February to July 2018. Information sheets were used to collect data about (a) personal demographics; (b) environmental conditions surrounding the parachute practice; and (c) parachute-related injuries. The incidence rate of injury was then calculated. Risk factors were examine